In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:
dataDF = pd.read_csv('all_features_v1_0.csv')
dataDF = dataDF.replace([np.inf,-np.inf], np.NaN)
for col in dataDF.columns[dataDF.isna().any()].tolist():
    dataDF[col] = dataDF.groupby(['date_start_period','STATEFP'])[col].transform(lambda x: x.fillna(x.mean()))

In [3]:
X = dataDF.drop(columns=["NAME","State_Name","STATEFP","COUNTYFP","GEOID","Unnamed: 0","target_date_4wk",
                         "target_date_3wk","target_date_2wk","date_end_period","date_start_period","date_end_lag","date_start_lag"])
y = dataDF[['LOG_DELTA_INC_RATE_T','LOG_DELTA_INC_RATE_T_14','LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T_28']]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=24)
X_train_wk1Pred = X_train.drop(columns=['LOG_DELTA_INC_RATE_T','LOG_DELTA_INC_RATE_T_14','LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T_28'])
X_test_wk1Pred = X_test.drop(columns=['LOG_DELTA_INC_RATE_T','LOG_DELTA_INC_RATE_T_14','LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T_28'])
X_train_wk2Pred = X_train.drop(columns=['LOG_DELTA_INC_RATE_T_14','LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T_28'])
X_test_wk2Pred = X_test.drop(columns=['LOG_DELTA_INC_RATE_T_14','LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T_28'])
X_train_wk3Pred = X_train.drop(columns=['LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T_28'])
X_test_wk3Pred = X_test.drop(columns=['LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T_28'])
X_train_wk4Pred = X_train.drop(columns=['LOG_DELTA_INC_RATE_T_28'])
X_test_wk4Pred = X_test.drop(columns=['LOG_DELTA_INC_RATE_T_28'])

In [5]:
y_train_wk1Pred = y_train.drop(columns=['LOG_DELTA_INC_RATE_T_14','LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T_28'])
y_test_wk1Pred = y_test.drop(columns=['LOG_DELTA_INC_RATE_T_14','LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T_28'])

y_train_wk2Pred = y_train.drop(columns=['LOG_DELTA_INC_RATE_T','LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T_28'])
y_test_wk2Pred = y_test.drop(columns=['LOG_DELTA_INC_RATE_T','LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T_28'])

y_train_wk3Pred = y_train.drop(columns=['LOG_DELTA_INC_RATE_T_14','LOG_DELTA_INC_RATE_T','LOG_DELTA_INC_RATE_T_28'])
y_test_wk3Pred = y_test.drop(columns=['LOG_DELTA_INC_RATE_T_14','LOG_DELTA_INC_RATE_T','LOG_DELTA_INC_RATE_T_28'])

y_train_wk4Pred = y_train.drop(columns=['LOG_DELTA_INC_RATE_T_14','LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T'])
y_test_wk4Pred = y_test.drop(columns=['LOG_DELTA_INC_RATE_T_14','LOG_DELTA_INC_RATE_T_21','LOG_DELTA_INC_RATE_T'])

In [6]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(RNNModel, self).__init__() # these are setting up the hyperparams for the RNN model
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True, nonlinearity='tanh') # using tanh function for activation
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, hn = self.rnn(x, h0.detach())
        out = self.fc(out[:, -1, :])  # Take last output
        return out

def train_and_predict(X_train, y_train, X_test, y_test):
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_train_scaled = scaler_x.fit_transform(X_train)
    X_test_scaled = scaler_x.transform(X_test)
    y_train_scaled = scaler_y.fit_transform(y_train)
    y_test_scaled = scaler_y.transform(y_test)

    model = RNNModel(input_dim=1, hidden_dim=512, num_layers=1, output_dim=1)
    criterion = torch.nn.MSELoss(reduction='mean')
    optimiser = torch.optim.Adam(model.parameters(), lr=0.0001)

    num_epochs = 100
    X_train_tensors = torch.Tensor(X_train_scaled).unsqueeze(2)  
    y_train_tensors = torch.Tensor(y_train_scaled)             

    X_test_tensors = torch.Tensor(X_test_scaled).unsqueeze(2)   
    y_test_tensors = torch.Tensor(y_test_scaled)                
    y_train_tensors = y_train_tensors.view(-1, 1)  
    y_test_tensors = y_test_tensors.view(-1, 1)   

    for epoch in range(num_epochs):
        model.train()
        optimiser.zero_grad()
        outputs = model(X_train_tensors)
        loss = criterion(outputs, y_train_tensors)  
        loss.backward()
        optimiser.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

    model.eval()
    y_test_pred = model(X_test_tensors)
    mse = mean_squared_error(y_test_tensors.numpy(), y_test_pred.detach().numpy())
    mae = mean_absolute_error(y_test_tensors.numpy(), y_test_pred.detach().numpy())
    print(f'RMSE: {np.root(mse)}')
    print(f'MAE: {mae}')

In [8]:
# takes a few hours to run :)
# train_and_predict(X_train_wk1Pred, y_train_wk1Pred, X_test_wk1Pred, y_test_wk1Pred)
# train_and_predict(X_train_wk2Pred, y_train_wk2Pred, X_test_wk2Pred, y_test_wk2Pred)
# train_and_predict(X_train_wk3Pred, y_train_wk3Pred, X_test_wk3Pred, y_test_wk3Pred)
# train_and_predict(X_train_wk4Pred, y_train_wk4Pred, X_test_wk4Pred, y_test_wk4Pred)